## Loading Data

In [1]:
import numpy as np
import itertools
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
)

## Visualization

In [2]:
import matplotlib.pyplot as plt

plt.imshow(train_data.data[0], cmap = 'gray')
plt.title('%i' % train_data.targets[0])
plt.show()
# plot multiple train_data
figure = plt.figure(figsize=(10,8))
cols, rows = 5, 5
for i in range(1,cols*rows+1):
    sample_idx = torch.randint(len(train_data),size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows,cols,i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(),cmap="gray")
plt.savefig('./visualization.png',dpi = 300)
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 1000x800 with 25 Axes>

## Building CNN

In [3]:
import torch.nn as nn
import torch.utils.data as utils_data
from torch import optim

In [4]:
#nnconv2d(a,f,b,c,d) hyper parameters a,b,c,d,e,f,g,h

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 1,
                out_channels = 10,
                kernel_size = 5,
                stride = 1,
                padding = 0
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels = 10,
                out_channels = 20,
                kernel_size = 5,
                stride = 1,
                padding = 0
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
        )
        # Fully Connected layer
        self.out = nn.Linear(320,10)  # 20*4*4 = 320. Original size is 28x28
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)    #flatten the output of conv2
        x = x.view(x.size(0),-1)
        output = self.out(x)
        return output, x
    
cnn = CNN()
print(cnn)
        

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=320, out_features=10, bias=True)
)


## Training

In [6]:
from torch.utils.data import DataLoader
loaders = {
    'train' : torch.utils.data.DataLoader(train_data,
                                         batch_size = 64,
                                         shuffle = True,
                                         num_workers = 1),
    'test' : torch.utils.data.DataLoader(test_data,
                                        batch_size = 64,
                                        shuffle = True,
                                        num_workers = 1)
}

## Loss Function & Optimizer

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr = 0.01)

## Train the Model

In [8]:
from torch.autograd import Variable
num_epochs = 10

def train(num_epochs, cnn, loaders):
    cnn.train()
    
    total_step = len(loaders['train'])
    for epoch in range (num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            b_x = Variable(images)
            b_y = Variable(labels)
            output = cnn(b_x)[0]
            
            loss = loss_func(output, b_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1)%100 == 0:
                print('Epoch[{}/{}], Step[{}/{}], Loss:{:.4f}'
                      .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                pass
        pass
    pass

train(num_epochs, cnn, loaders)

Epoch[1/10], Step[100/938], Loss:0.1279
Epoch[1/10], Step[200/938], Loss:0.3253
Epoch[1/10], Step[300/938], Loss:0.1084
Epoch[1/10], Step[400/938], Loss:0.0985
Epoch[1/10], Step[500/938], Loss:0.0300
Epoch[1/10], Step[600/938], Loss:0.0366
Epoch[1/10], Step[700/938], Loss:0.0723
Epoch[1/10], Step[800/938], Loss:0.0653
Epoch[1/10], Step[900/938], Loss:0.0808
Epoch[2/10], Step[100/938], Loss:0.1281
Epoch[2/10], Step[200/938], Loss:0.1198
Epoch[2/10], Step[300/938], Loss:0.0419
Epoch[2/10], Step[400/938], Loss:0.0750
Epoch[2/10], Step[500/938], Loss:0.0087
Epoch[2/10], Step[600/938], Loss:0.0122
Epoch[2/10], Step[700/938], Loss:0.0738
Epoch[2/10], Step[800/938], Loss:0.0389
Epoch[2/10], Step[900/938], Loss:0.0047
Epoch[3/10], Step[100/938], Loss:0.0427
Epoch[3/10], Step[200/938], Loss:0.0091
Epoch[3/10], Step[300/938], Loss:0.0225
Epoch[3/10], Step[400/938], Loss:0.0548
Epoch[3/10], Step[500/938], Loss:0.0898
Epoch[3/10], Step[600/938], Loss:0.0681
Epoch[3/10], Step[700/938], Loss:0.0106


## Test Model

In [9]:
def test():
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in loaders['test']:
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            pass
    print('Test Accuracy of the model on the 10000 test images: %.2f' %accuracy)
    
    pass
test()
sample = next(iter(loaders['test']))
imgs, lbls = sample
actual_number = lbls[:20].numpy()
test_output, last_layer = cnn(imgs[:20])
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
print(f'Prediction number: {pred_y}')
print(f'Actual number: {actual_number}')

Test Accuracy of the model on the 10000 test images: 1.00
Prediction number: [6 6 6 2 0 8 6 8 1 5 9 1 6 9 7 8 1 8 8 0]
Actual number: [6 6 6 2 0 8 6 8 1 9 9 1 6 9 7 8 1 8 8 0]
